# Kanren

As you should know by now, Python is not really a logical programming language. Before we start logic programming in Python, we need to install Kanren, a Python package that enables logic programming in Python. 

## 1. Installation

First install the library using the following command:

In [1]:
pip install kanren

     |████████████████████████████████| 55 kB 4.3 MB/s eta 0:00:011
  Created wheel for kanren: filename=kanren-0.2.3-py3-none-any.whl size=15860 sha256=02c8ed4df3ac9a704ccb0ed98dc97f0a78423e45fde3a91f6930b9503db00309
  Stored in directory: /home/yori/.cache/pip/wheels/69/e7/58/f138e1a8e1b6a717490d4c69ed313e46b459fb5bde2a100b9b
Successfully built kanren
Note: you may need to restart the kernel to use updated packages.


## 2. Ask Kanren

Kanren enables the expression of relations and the search for values which satisfy them. The following code is the *Hello world!* of logic programming.

In [4]:
from kanren import run, eq, membero, var, conde # some of these imports will be used later

x = var() # declare a variable x
result = run(1, x, eq(x, 5)) # ask kanren 1 value x, such that x equals 5

print(result)

(5,)


Multiple variables and multiple goals can be used simultaneously. The
following code asks for a number x such that `x == z` and `z == 3`

In [5]:
z = var('z') # you may also, optionally, pass a token name for a variable...
print(z)     # and print it

run(0, x, eq(x, z), eq(z, 3)) # ask kanren for all values (just say 0), immediately print the result, you may omit the print statement

~z


(3,)

The following code asks for a number, x, such that `(1, 2) == (1, x)` holds. The variable x was already declared above.

In [6]:
run(0, x, eq((1, 2), (1, x)))

(2,)

The above examples use `eq`, a *goal constructor* to state that two expressions
are equal. Other goal constructors exist such as `membero(item, coll)` which
states that `item` is a member of `coll`, a collection.

The following example uses `membero` twice to ask for the values of x,
such that x is a member of `(1, 2, 3)` and that x is a member of `(2, 3, 4)`.

In [7]:
run(0, x, membero(x, (1, 2, 3)),  # x is a member of (1, 2, 3)
          membero(x, (2, 3, 4)))  # x is a member of (2, 3, 4)

(2, 3)

## 3. Facts

Kanren stores data as facts. Facts are relationships (between terms). In the next example *state* and *border* are two relations.

In [8]:
from kanren import fact, facts, Relation

state = Relation()
border = Relation()

fact(state, "washington")   # declare one fact
fact(state, "oregon")
facts(state, "idaho",       # use facts to declare more facts in one statement
             "california")

fact(border, "washington", "oregon") # one fact
facts(border, ("washington", "idaho"), # facts to declare multiple facts
              ("oregon", "california"))

run(0, x, border(x, "oregon")) # ask Kanren which state borders oregon, note that it outputs only washington

('washington',)

## 4. Rules

Use the Python `def` statement to declare a rule. x and y are adjacent if x borders y.

In [9]:
def adjacent(x, y):
    return border(x, y)

result = run(0, x, adjacent("washington", "oregon"))
print('yes' if len(result) else 'no') # print yes if the length of the result is > 0, no otherwise

result = run(0, x, adjacent("oregon", "washington"))
print('yes' if len(result) else 'no')

yes
no


Use `conde`, a goal constructor for logical *and* and *or*.  __Use brackets for *and*, square brackets for *or*. Please note the position of the brackets! They are different for the normal and square brackets!__

```python
def adjacent(x, y):
    return conde([border(x, y)], [border(y, x)]) # x is adjacent to y if x borders y OR y borders x

def grandparent(x, y):
    z = var()
    return conde((parent(x, z), parent(z, y)))   # x is the grandparent of y if x is the parent of z AND z is the parent of y

```    

In [10]:
def adjacent(x, y):
    return conde([border(x, y)], [border(y, x)])

result = run(0, x, adjacent("washington", "oregon"))
print('yes' if len(result) else 'no')

result = run(0, x, adjacent("oregon", "washington"))
print('yes' if len(result) else 'no')

yes
yes


In [11]:
print(run(0, x, adjacent(x, "oregon"))) # both states are found now!
print(run(0, x, adjacent(x, "washington"))) # both states are found now!

('washington', 'california')
('idaho', 'oregon')
